# Elastic constants

We compute *clamped-ion* elastic constants of a crystal using
the algorithmic differentiation density-functional perturbation theory (AD-DFPT) approach
as introduced in [^SPH25].

[^SPH25]:
    Schmitz, N. F., Ploumhans, B., & Herbst, M. F. (2025)
    *Algorithmic differentiation for plane-wave DFT: materials design, error control and learning model parameters.*
    [arXiv:2509.07785](https://arxiv.org/abs/2509.07785)

We consider a crystal in its equilibrium configuration, where all atomic forces
and stresses vanish.  Homogeneous strains $η$ are then applied
relative to this relaxed structure.
The elastic constants are derived from the stress-strain relationship.
In [Voigt notation](https://en.wikipedia.org/wiki/Voigt_notation),
the stress $\sigma$ and strain $\eta$ tensors are represented as 6-component vectors.
The elastic constants $C$ are then given by
the Jacobian of the stress with respect to strain, forming a $6 \times 6$ matrix
$$
  C = \frac{\partial \sigma}{\partial \eta}.
$$

The sparsity pattern of the matrix $C$ follows from crystal symmetry
and is tabulated in standard references (eg. Table 9 in [^Nye1985]).
This sparsity can be used a priori to reduce the number of strain patterns
that need to be probed to extract all independent components of $C$.
For example, cubic crystals have only three independent elastic constants
$C_{11}$, $C_{12}$ and $C_{44}$, with the pattern
$$
C = \begin{pmatrix}
  C_{11} & C_{12} & C_{12} & 0      & 0      & 0 \\
  C_{12} & C_{11} & C_{12} & 0      & 0      & 0 \\
  C_{12} & C_{12} & C_{11} & 0      & 0      & 0 \\
  0      & 0      & 0      & C_{44} & 0      & 0 \\
  0      & 0      & 0      & 0      & C_{44} & 0 \\
  0      & 0      & 0      & 0      & 0      & C_{44} \\
\end{pmatrix}.
$$
Thus we can just choose a suitable strain pattern $\dot{\eta} = (1, 0, 0, 1, 0, 0)^\top$,
such that $C\dot{\eta} = (C_{11}, C_{12}, C_{12}, C_{44}, 0, 0)^\top$. That is,
for cubic crystals like diamond silicon we obtain all independent elastic
constants from a single Jacobian-vector product on the stress-strain function.

[^Nye1985]:
     Nye, J. F. (1985).
     *Physical Properties of Crystals*. Oxford University Press.
     Comment: Since the elastic tensor transforms equivariantly under rotations,
     its numerical components depend on the chosen Cartesian coordinate frame.
     These tabulated patterns assume a standardized orientation of the structure
     with respect to conventional crystallographic axes.

This example computes the *clamped-ion* elastic tensor, keeping internal
atomic positions fixed under strain.  The *relaxed-ion* tensor includes
additional corrections from internal relaxations, which can be obtained
from first-order atomic displacements in DFPT (see [^Wu2005]).

[^Wu2005]:
    Wu, X., Vanderbilt, D., & Hamann, D. R. (2005).
    *Systematic treatment of displacements, strains, and electric fields in density-functional perturbation theory.*
    [Physical Review B, 72(3), 035105](https://doi.org/10.1103/PhysRevB.72.035105).

In [1]:
import Pkg
Pkg.activate(joinpath(@__DIR__, ".."))

using DFTK
using PseudoPotentialData
using LinearAlgebra
using ForwardDiff
using DifferentiationInterface
using AtomsBuilder
using Unitful
using UnitfulAtomic

  Activating project at `~/git/tutorial-cecam-workshop-dftk-2025`


In [2]:
pseudopotentials = PseudoFamily("dojo.nc.sr.pbe.v0_4_1.standard.upf")
a0_pbe = 10.33u"bohr"  # Equilibrium lattice constant of silicon with PBE
model0 = model_DFT(bulk(:Si; a=a0_pbe); pseudopotentials, functionals=PBE())

Ecut = recommended_cutoff(model0).Ecut
kgrid = [4, 4, 4]
tol = 1e-6

function symmetries_from_strain(model0, voigt_strain)
    lattice = DFTK.voigt_strain_to_full(voigt_strain) * model0.lattice
    model = Model(model0; lattice, symmetries=true)
    model.symmetries
end

strain_pattern = [1., 0., 0., 1., 0., 0.];  # recovers [c11, c12, c12, c44, 0, 0]

For elastic constants beyond the bulk modulus, symmetry-breaking strains
are required. That is, the symmetry group of the crystal is reduced.
Here we simply precompute the relevant subgroup by applying the automatic
symmetry detection (spglib) to the finitely perturbed crystal.

In [3]:
symmetries_strain = symmetries_from_strain(model0, 0.01 * strain_pattern)


function stress_from_strain(model0, voigt_strain; symmetries, Ecut, kgrid, tol)
    lattice = DFTK.voigt_strain_to_full(voigt_strain) * model0.lattice
    model = Model(model0; lattice, symmetries)
    basis = PlaneWaveBasis(model; Ecut, kgrid)
    scfres = self_consistent_field(basis; tol)
    DFTK.full_stress_to_voigt(compute_stresses_cart(scfres))
end

stress_fn(voigt_strain) = stress_from_strain(model0, voigt_strain;
                                             symmetries=symmetries_strain,
                                             Ecut, kgrid, tol)
stress, (dstress,) = value_and_pushforward(stress_fn, AutoForwardDiff(),
                                           zeros(6), (strain_pattern,));

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -8.453520355075                   -0.94    5.3    5.08s
  2   -8.455630775648       -2.68       -1.77    1.0    2.27s
  3   -8.455776321229       -3.84       -2.88    1.9    640ms
  4   -8.455790066922       -4.86       -3.27    3.0    240ms
  5   -8.455790174436       -6.97       -3.78    1.2    153ms
  6   -8.455790186833       -7.91       -4.92    1.4    141ms
  7   -8.455790187706       -9.06       -5.52    2.9    206ms
  8   -8.455790187713      -11.18       -5.97    1.6    149ms
  9   -8.455790187713      -12.37       -6.51    1.3    151ms
Solving response problem
Iter  Restart  Krydim  log10(res)  avg(CG)  Δtime   Comment
----  -------  ------  ----------  -------  ------  ---------------
                                      66.1   4.23s  Non-interacting
   1        0       1        0.11     50.0   8.05s  
   2        0       2       -0.70     45.9   

We can inspect the stress to verify it is small (close to equilibrium):

In [4]:
stress

6-element StaticArraysCore.SVector{6, Float64} with indices SOneTo(6):
 -2.3121110372192445e-5
 -2.31211337804883e-5
 -2.3121133780488326e-5
 -1.1930420092768635e-10
  0.0
  0.0

The response of the stress to `strain_pattern` contains the elastic constants
in atomic units, with the expected pattern $(C_{11}, C_{12}, C_{12}, C_{44}, 0, 0)$:

In [5]:
dstress

6-element StaticArraysCore.SVector{6, Float64} with indices SOneTo(6):
 0.0053199437119925705
 0.0020248998061615977
 0.0020248998061615977
 0.003351909209025156
 0.0
 0.0

Convert to GPa:

In [6]:
println("C11: ", uconvert(u"GPa", dstress[1] * u"hartree" / u"bohr"^3))
println("C12: ", uconvert(u"GPa", dstress[2] * u"hartree" / u"bohr"^3))
println("C44: ", uconvert(u"GPa", dstress[4] * u"hartree" / u"bohr"^3))

C11: 156.51814745572818 GPa
C12: 59.57460898118596 GPa
C44: 98.61657345241198 GPa
 GPa
C12: 59.57460898118596 GPa
C44: 98.61657345241198 GPa


These results can be compared directly to finite differences of the stress_fn:

In [7]:
h = 1e-3
dstress_fd = (stress_fn(h * strain_pattern) - stress_fn(-h * strain_pattern)) / 2h
println("C11 (FD): ", uconvert(u"GPa", dstress_fd[1] * u"hartree" / u"bohr"^3))
println("C12 (FD): ", uconvert(u"GPa", dstress_fd[2] * u"hartree" / u"bohr"^3))
println("C44 (FD): ", uconvert(u"GPa", dstress_fd[4] * u"hartree" / u"bohr"^3))

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -8.453543098614                   -0.94    5.4    656ms
  2   -8.455636235179       -2.68       -1.77    1.0    333ms
  3   -8.455781729855       -3.84       -2.89    1.8    435ms
  4   -8.455795193357       -4.87       -3.26    3.1    986ms
  5   -8.455795301917       -6.96       -3.78    1.1    332ms
  6   -8.455795314332       -7.91       -4.82    1.4    410ms
  7   -8.455795315191       -9.07       -5.69    2.8    616ms
  8   -8.455795315198      -11.15       -6.41    2.0    376ms
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -8.453493228720                   -0.94    5.4    388ms
  2   -8.455624490153       -2.67       -1.77    1.0    163ms
  3   -8.455770064867       -3.84       -2.89    1.9    219ms
  4   -8.455782148007       -4.92       -3.31    3.1    300ms
  5   -8.4

Here are AD-DFPT results from increasing discretization parameters:

| Ecut | kgrid         | c11 [GPa]   | c12 [GPa]  | c44 [GPa]  |
|------|---------------|-------:|------:|-------:|
| 18   | [4, 4, 4]     | 156.51 | 59.57 |  98.61 |
| 18   | [8, 8, 8]     | 153.53 | 56.90 | 100.07 |
| 24   | [8, 8, 8]     | 153.26 | 56.82 |  99.97 |
| 24   | [14, 14, 14]  | 153.03 | 56.71 | 100.09 |

For comparison, Materials Project for PBE *relaxed-ion* elastic constants of
silicon [mp-149](https://next-gen.materialsproject.org/materials/mp-149):
c11 = 153 GPa, c12 = 57 GPa, c44 = 74 GPa.
Note the discrepancy in c44, which is due to us not yet including
ionic relaxation in this example.